In [1]:
!pip install ultralytics
!pip install thop
!pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.5/974.5 kB 26.9 MB/s eta 0:00:0000:01


In [2]:
from ultralytics import YOLO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from pathlib import Path
import time
import glob
from thop import profile
import torch  # Added for FLOPs

# Load YOLOv12 model (medium version)
model = YOLO("yolo12m.pt")

# Training configuration
project = "MoonCraterTraining"
name = "yolo12m_run_v23"  # Match your run
output_dir = Path(project) / name

# Train with improved settings
results = model.train(
    data="/kaggle/input/yolomoon1/yolomoon/data.yaml",
    epochs=200,  # More epochs for small dataset
    patience=50,  # Longer patience to push mAP
    batch=8,
    imgsz=896,  # Larger for small craters
    amp=True,
    val=True,
    save_period=5,
    project=project,
    name=name,
    mosaic=1.0,
    mixup=0.3,  # Upped for variety
    hsv_h=0.03,
    hsv_s=0.8,
    hsv_v=0.4,
    degrees=20.0,
    scale=0.7,
    shear=5.0,
    lr0=0.0003,  # Lower LR
    cos_lr=True,
    warmup_epochs=5,  # Gradual start
)

# --- Accuracy Chart ---
metrics_file = output_dir / "results.csv"
if metrics_file.exists():
    df = pd.read_csv(metrics_file)
    plt.figure(figsize=(10, 6))
    plt.plot(df["metrics/mAP50(B)"], label="mAP@0.5", marker="o")
    plt.plot(df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95", marker="o")
    plt.plot(df["train/box_loss"], label="Train Box Loss", marker="x")
    plt.plot(df["val/box_loss"], label="Val Box Loss", marker="x")
    plt.xlabel("Epoch")
    plt.ylabel("Metric")
    plt.title("Accuracy Metrics Over Epochs")
    plt.legend()
    plt.grid(True)
    accuracy_chart_path = output_dir / "accuracy_chart.png"
    plt.savefig(accuracy_chart_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Accuracy chart saved: {accuracy_chart_path}")

# --- Latency Chart ---
image_extensions = ["*.png", "*.jpg", "*.jpeg"]
sample_img = None
for ext in image_extensions:
    for folder in ["/kaggle/input/yolomoon1/yolomoon/val/images", "/kaggle/input/yolomoon1/yolomoon/train/images"]:
        images = list(Path(folder).glob(ext))
        if images:
            sample_img = images[0]
            break
    if sample_img:
        break

if sample_img:
    latencies = []
    num_trials = 10
    for _ in range(num_trials):
        start_time = time.time()
        model.predict(sample_img, imgsz=896, verbose=False)  # Match imgsz
        latency_ms = (time.time() - start_time) * 1000
        latencies.append(latency_ms)

    plt.figure(figsize=(8, 5))
    plt.plot(range(num_trials), latencies, marker="o", color="blue")
    plt.axhline(np.mean(latencies), color="red", linestyle="--", label=f"Mean: {np.mean(latencies):.2f} ms")
    plt.xlabel("Trial")
    plt.ylabel("Inference Time (ms)")
    plt.title("Inference Latency per Image")
    plt.legend()
    plt.grid(True)
    latency_chart_path = output_dir / "latency_chart.png"
    plt.savefig(latency_chart_path, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"Latency chart saved: {latency_chart_path}")
else:
    print("No sample image found for latency test. Skipping latency chart.")

# --- FLOPs Chart ---
model.model.eval()
input_tensor = torch.randn(1, 3, 896, 896).to(model.device)  # Match imgsz
flops_val, _ = profile(model.model, inputs=(input_tensor,))  # FLOPs in ops
flops_gflops = flops_val / 1e9  # Convert to GFLOPs

plt.figure(figsize=(6, 4))
plt.bar(["yolo12m"], [flops_gflops], color="green")
plt.ylabel("GFLOPs")
plt.title("Model Computational Complexity")
plt.grid(True, axis="y")
flops_chart_path = output_dir / "flops_chart.png"
plt.savefig(flops_chart_path, dpi=300, bbox_inches="tight")
plt.close()
print(f"FLOPs chart saved: {flops_chart_path}")

# Save summary
summary = {
    "mAP@0.5": df["metrics/mAP50(B)"].max() if metrics_file.exists() else 0,
    "mAP@0.5:0.95": df["metrics/mAP50-95(B)"].max() if metrics_file.exists() else 0,
    "Mean Latency (ms)": np.mean(latencies) if latencies else None,
    "FLOPs (GFLOPs)": flops_gflops,
}
pd.DataFrame([summary]).to_csv(output_dir / "training_summary.csv", index=False)
print(f"Summary saved: {output_dir / 'training_summary.csv'}")

# Export to ONNX
exported_model_path = model.export(format="onnx")
print(f"Model exported successfully: {exported_model_path}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 39.0M/39.0M [00:00<00:00, 348MB/s]


Ultralytics 8.3.107 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=yolo12m.pt, data=/kaggle/input/yolomoon1/yolomoon/data.yaml, epochs=200, time=None, patience=50, batch=8, imgsz=896, save=True, save_period=5, cache=False, device=None, workers=8, project=MoonCraterTraining, name=yolo12m_run_v23, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=T

100%|██████████| 755k/755k [00:00<00:00, 26.0MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1      1856  ultralytics.nn.modules.conv.Conv             [3, 64, 3, 2]                 
  1                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  2                  -1  1    111872  ultralytics.nn.modules.block.C3k2            [128, 256, 1, True, 0.25]     
  3                  -1  1    590336  ultralytics.nn.modules.conv.Conv             [256, 256, 3, 2]              
  4                  -1  1    444928  ultralytics.nn.modules.block.C3k2            [256, 512, 1, True, 0.25]     
  5                  -1  1   2360320  ultralytics.nn.modules.conv.Conv             [512, 512, 3, 2]              
  6                  -1  2   2689536  ultralytics.nn.modules.block.A2C2f           [512, 512, 2, True, 4]        
  7                  -1  1   2360320  ultralytics

...


200 epochs completed in 0.485 hours.
Optimizer stripped from MoonCraterTraining/yolo12m_run_v23/weights/last.pt, 40.8MB
Optimizer stripped from MoonCraterTraining/yolo12m_run_v23/weights/best.pt, 40.8MB

Validating MoonCraterTraining/yolo12m_run_v23/weights/best.pt...
Ultralytics 8.3.107 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv12m summary (fused): 169 layers, 20,105,683 parameters, 0 gradients, 67.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.24it/s]


                   all         12        767       0.62      0.554      0.599      0.249


/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.10/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


Speed: 0.2ms preprocess, 24.8ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to MoonCraterTraining/yolo12m_run_v23
Accuracy chart saved: MoonCraterTraining/yolo12m_run_v23/accuracy_chart.png
Latency chart saved: MoonCraterTraining/yolo12m_run_v23/latency_chart.png
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
FLOPs chart saved: MoonCraterTraining/yolo12m_run_v23/flops_chart.png
Summary saved: MoonCraterTraining/yolo12m_run_v23/training_summary.csv
Ultralytics 8.3.107 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.00GHz)

PyTorch: starting from 'MoonCraterTraining/yolo12m_run_v23/weights/best.pt' with input shape (1, 3, 896, 896) BCHW and output shape(s) (1, 5, 16464) (38.9 MB)
requirements: Ultralytics requirements ['onnxslim', 'onnxruntime-gpu'] not found, attempting A

In [ ]:
cat /kaggle/input/yolomoon1/yolomoon/data.yaml

In [ ]:
import shutil

# Define the folder you want to download
folder_path = "/kaggle/working/runs"  # Replace with your folder path
zip_file_path = "/kaggle/working/my_model_output.zip"

# Zip the folder
shutil.make_archive(zip_file_path.replace(".zip", ""), 'zip', folder_path)

print(f"Folder zipped successfully: {zip_file_path}")

In [ ]:
from IPython.display import FileLink

FileLink(zip_file_path)

In [ ]:
import ultralytics
print(ultralytics.__version__)